### Project: explore customer and track sales in chinook digital store
The objective for this project is to analyse data from the chinook database, which has information about a fictional digital music shop - kind of like a mini-iTunes store.

The chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables, click [here](https://github.com/lerocha/chinook-database) to visit the github repository.


In [ ]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Preview data tables in the chinook db file.

In [ ]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

### Popular music genres in the US

The most popular music genre for customers purchasing music from the US is `Rock`. The `Rock` genre has 53% of sales from customer purchases while the `Alternative & Punk` genre has 12% of sales followed closely behind by `Metal` genre which has 11.7% of sales. Much further from the Top 3 genres in fourth place is the `R&B/Soul` music genre which has 5% of total sales.

Based on global sales of tracks across different genres in the US, Chinook record store should purchase albums by the first three artists:

- `Red Tone	- Punk`: Punk genre is ranked 2nd overall in sales (Recommended) 
- `Slim Jim Bites - Blues`: Blues genre is ranked 5th overall (Recommended)
- `Meteor and the Girls - Pop`: Pop genre is ranked joint 7th overall with `Latin` music (Recommended)

The below artist narrowly missed the recommended shortlist, though hip-hop/rap might be ranked 8th overall in customer purchases from the US, the genre was only 0.19 per-cent off the 7th position.
- `Regal - Hip-Hop`: Hip Hop/Rap genre is ranked 8th overall

In [ ]:
%%sql
WITH popular_usa_genres AS 
   (
    SELECT il.*,
           c.country 
    FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )
SELECT g.name genre, 
       COUNT(pug.invoice_line_id) tracks_purchased, 
       round(CAST(COUNT(pug.invoice_line_id) AS FLOAT) / (
           SELECT COUNT(*)
           FROM popular_usa_genres
       ) * 100, 2) percentage_purchased
FROM popular_usa_genres pug
INNER JOIN track t on t.track_id = pug.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY g.genre_id
ORDER BY tracks_purchased desc;

### Total customer sales by sales support agent

The below data displays total dollar amount of sales assigned to each Sales Support Agent within the organisation.

The highest performing Sales Agent employee is Jane Peacock with 36.77% of total sales followed closely by Margaret Park with 33.63%. Steve Johnson has the least percentage of sales recorded with 29.6%, roughly 7% off the top performing employee. 

Based on the hire date for these employees, the data suggests that the employee performance in sales seems to have some correlation with the dates at which they had joined the company. Jane was the first sales support agent amongst the three to be hired, while Steve's percentage sales amount could be due to being hired at least five months later than the others.

In [ ]:
%%sql
WITH rep_sales AS
    (
        SELECT
        c.customer_id,
        round(sum(i.total), 2) rep_sales_total,
        c.support_rep_id
        FROM customer c
        INNER JOIN invoice i on i.customer_id = c.customer_id
        GROUP BY i.customer_id
    )
SELECT 
    e.first_name || ' ' || e.last_name employee_name,
    SUM(rs.rep_sales_total) rep_sales_total,
    round(CAST(SUM(rs.rep_sales_total) AS FLOAT) /
    (
        SELECT SUM(rep_sales_total) 
        FROM rep_sales
    ) * 100, 2) percentage_sales,
    e.hire_date
FROM rep_sales rs
INNER JOIN employee e on e.employee_id = rs.support_rep_id
WHERE e.title in ('Sales Support Agent')
GROUP BY employee_name;


### Sales data for customers per country

The below shows sales data, for each country by calculating:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value
Because there are a number of countries with only one customer, these customers will be grouped in 'Other'.


The US might have the most customer sales overall, but the average order value per customer is only `$80` (the second lowest total amount across the countries excluding those with only one customer grouped in 'other'). The data suggests that there might be opportunity for higher sales in Czech Republic and Portugal which have `$136.62` and `$92.57` average order values per customer, though this data should be used with caution due to the limited sample size being used for the analysis.

In [ ]:
%%sql
WITH customer_sales AS 
   (
    SELECT il.*,
           CASE
               WHEN (
                     SELECT count(*)
                     FROM customer
                     where country = c.country
                    ) = 1 THEN "Other"
               ELSE c.country
           END AS country, 
           i.total,
           c.customer_id
    FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
   )
SELECT 
    country, 
    count(distinct(customer_id)) total_customers, 
    round(SUM(unit_price), 2) total_sales,
    round(SUM(unit_price) / count(distinct customer_id), 2) avg_sales_per_customer,
    round(SUM(unit_price) / count(distinct invoice_id), 2) avg_order_value
FROM customer_sales cs
GROUP BY country
ORDER BY total_sales DESC;

### Popular artists used in the most playlists
The most popular artist used in playlists is `Iron Maiden` while `U2` and `Metallica` come in 2nd and 3rd place overall.

In [ ]:
%%sql
SELECT 
    a.artist_id, 
    a.name artist_name, 
    count(a.name) tracks_in_playlist
FROM playlist_track pt
INNER JOIN track t on t.track_id = pt.track_id
INNER JOIN album ab on ab.album_id = t.album_id
INNER JOIN artist a on a.artist_id = ab.artist_id
GROUP BY a.name
ORDER BY tracks_in_playlist DESC
LIMIT 10;

### Total count of purchased and non-purchased tracks 

There are a total of `3503` tracks stored in the `track` table, of this, customers have purchased `1806` tracks from the library. There are still `1697` tracks yet to be purchased. 

In [ ]:
%%sql
WITH total_tracks AS 
    (
        SELECT 
            count(distinct track_id) total_tracks, 
    (
        SELECT 
            count(distinct track_id) 
        FROM invoice_line) purchased
        FROM track
    )
SELECT tt.*, 
       (tt.total_tracks - tt.purchased) non_purchased 
FROM total_tracks tt;

Displayed below are some of the tracks which are *yet* to be purchased from the store.

In [ ]:
%%sql
SELECT 
    t.*
FROM track t 
WHERE t.track_id not in (
    SELECT 
        il.track_id 
    FROM invoice_line il
    )
LIMIT 10;